In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import pickle

import lightgbm as lgb

In [2]:
#Creating tests folder
import os
if not os.path.isdir("tests") :
    os.mkdir("tests")

In [3]:
"""
This function looks at the data in every column and sets the most efficient data type to it - memory wise. 
"""
def reduce_mem_usage(props):    
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [4]:
"""
Loading Kaggle data.
"""
calendar = pd.read_csv('calendar.csv')
sales_train_validation = pd.read_csv('sales_train_validation.csv')
sample_submission = pd.read_csv('sample_submission.csv')
sell_prices = pd.read_csv('sell_prices.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Aviv\\Desktop\\work\\kaggle\\2020\\M5\\accuracy\\calendar.csv'

In [ ]:
# get features from calendar data set to train DataFrame
MONTH_START = 8
train = calendar.drop(["date", "weekday"], axis = 1).copy()
train["mday"] = calendar.date.str[MONTH_START:].astype("int")
train["special_day"] = np.where((train["event_name_1"].astype("str") != "nan") |
                                (train["event_name_2"].astype("str") != "nan") , True, False)
train = train.drop(["event_name_1", "event_type_1", "event_name_2", "event_type_2"], axis = 1)

In [ ]:
"""
From classic time series table (every column is a day, every row a product) 
To a train ready Dataframe - 
every row is a product at a given day 
total number of rows = days * products
"""
DAYS_COUNT = 1913

# month, day,state, department, price, sold
sales_train_validation["day"] = 0
sales_train_validation["sold"] = 0

# take every row - duplicate id 1913 times (once for every day, add the day number to id)
# take all d_1.. d_1913 matrix and flatten it - rotate it so it is a column
# now you have new id and sold items - so you know item, cat, dept, state and price in every day.

# sales_id = sales_train_validation["id"].copy()
new_id = sales_train_validation["id"].loc[np.repeat(sales_train_validation.index.values, DAYS_COUNT)].copy()
keep_id = new_id.copy()

days_array = list(range(1,DAYS_COUNT + 1))
number_of_items = sales_train_validation.shape[0]
all_days = pd.Series(days_array * number_of_items)

new_id.index = all_days.index
keep_id.index = new_id.index
new_id = new_id.astype("str") + all_days.astype("str")

days = [col for col in sales_train_validation if "d_" in col]
only_days_sales = np.array(sales_train_validation[days].copy()).flatten()

product_sales = pd.DataFrame({"id": new_id, "sales": only_days_sales})

product_sales["product_id"] = keep_id
product_sales["d"] ="d_" + all_days.astype("str")

product_sales = product_sales.merge(train, how="left", on="d")
product_sales = product_sales.merge(sales_train_validation[["id", "dept_id", "state_id","item_id","store_id"]], how="left", left_on="product_id", right_on="id")

product_sales = product_sales.rename(columns = {"id_x": "id"})
product_sales = product_sales.drop("id_y", axis = 1)

product_sales = product_sales.merge(sell_prices, how="left", on = ["store_id", "item_id", "wm_yr_wk"])
# product_sales = product_sales.dropna()
# product_sales["sell_price"] = product_sales["sell_price"].astype("int")

product_sales = product_sales.drop("sell_price", axis = 1)

In [ ]:
product_sales.head(1)

In [ ]:
"""
Label Encode department and state.
"""
le_dept = LabelEncoder()
le_state = LabelEncoder()

categorical_cols = ["dept_id", "state_id"]

product_sales["dept_id"] = le_dept.fit_transform(product_sales["dept_id"])
product_sales["state_id"] = le_state.fit_transform(product_sales["state_id"])

pickle.dump(le_dept, open("le_dept.pkl", "wb"))
pickle.dump(le_state, open("le_state.pkl", "wb"))


In [ ]:
"""
Save train-ready DataFrame, with the best data types.
"""
product_sales, _ = reduce_mem_usage(product_sales)
pickle.dump(product_sales, open("init_train.pkl", "wb"))

In [ ]:
"""
For later test building, split the id column to it's components.
"""
splitted = sample_submission["id"].str.split("_")
splitted = pd.DataFrame.from_dict(dict(zip(splitted.index, splitted.values)), orient="index")
splitted["id"] = sample_submission["id"]
splitted["dept_id"] = splitted[0].astype("str") + "_" + splitted[1].astype("str")
splitted["state_id"] = splitted[3]
splitted["store_id"] = splitted[3].astype("str") + "_" + splitted[4].astype("str")
splitted["item_id"] = splitted["dept_id"] + "_" + splitted[2]
splitted.head(1)

In [ ]:
"""
Simpilfy days with "events" into one feature - special_day
"""
days_df = calendar.drop(["date", "weekday"], axis = 1).copy()
days_df["special_day"] = np.where((days_df["event_name_1"].astype("str") != "nan") |
                                (days_df["event_name_2"].astype("str") != "nan") , True, False)
days_df = days_df.drop(["event_name_1", "event_type_1", "event_name_2", "event_type_2",
                       "month", "d"], axis = 1)

In [ ]:
"""
For test sets, get the last record date.
this will allow the notebook to work no matter the dates.
"""
last_record =  product_sales.tail(1).copy()
cur_wm_yr_wk = last_record["wm_yr_wk"].values[0]
cur_wday = product_sales.tail(1)["wday"].values[0]
cur_month = product_sales.tail(1)["month"].values[0]
cur_mday = product_sales.tail(1)["mday"].values[0]

In [ ]:
"""
Building a test set for every day.
"""

month_days = {1: 31, 2: 29, 3 :31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12:31}

test =splitted[["id", "dept_id", "state_id", "item_id", "store_id"]].copy()
test_days = []
cur_mday = cur_mday + 1
cur_wday = (cur_wday + 1) % 8

for i in range(1,29):
    
    test_days.append(test.copy())
    if cur_wday == 0: # next week
        cur_wday = 1
        cur_wm_yr_wk = cur_wm_yr_wk + 1

    if cur_mday > month_days[cur_month]: # we are in the next month
        cur_month = (cur_month + 1) % 13
        if cur_month == 0:
            cur_month = 1
        
        cur_mday = 1 # because the first thing we do in the loop is cur_mday + 1
    
    test_days[-1]["mday"] = cur_mday 
    test_days[-1]["wday"] = cur_wday 
    test_days[-1]["month"] = cur_month 
    test_days[-1]["wm_yr_wk"] = cur_wm_yr_wk 
    
    test_days[-1] = test_days[-1].merge(days_df, on=["wm_yr_wk", "wday"], how = "left") 
    test_days[-1] = test_days[-1].merge(sell_prices, on = ["store_id", "item_id", "wm_yr_wk"], how = "left")
    
    cur_mday = cur_mday + 1
    cur_wday = (cur_wday + 1) % 8

In [ ]:
for i, test in enumerate(test_days):
    test = test.drop("sell_price", axis = 1)
    pickle.dump(test, open('tests/test_{}.pkl'.format(i), "wb"))